In [422]:
import pandas as pd
import yfinance as yf
import bs4 as bs
import pickle
import requests
import numpy as np


In [423]:
# function to get the tickers using webscrapping from wikipedia
def sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    
    tickers=[s[:-1] for s in tickers] #adjust strings
    
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

tickers=sp500_tickers()

In [81]:
#function to download data from Yahoo Finance 
def get_data(tickers,date_start,date_end):
    df = pd.DataFrame(columns=tickers)
    for ticker in tickers:
        df[ticker]=yf.download(ticker,date_start,date_end)['Adj Close']
    return df

#specify ticker and dates
dfget_data(tickers,"2010-01-01","2019-01-01")

In [295]:
df_clean=df.dropna(axis=1)
tickers_clean=df_clean.columns

In [331]:
df_clean_returns=df_clean.pct_change()
vol_df=df_clean_returns.std()*np.sqrt(250)

In [480]:
class Returns:
    def __init__(self,data,tickers):
        self.data=data
        self.tickers=tickers
        
    def annualized_return(self,year):
        total_return = pd.DataFrame(columns=self.tickers)
        for ticker in self.data:
            total_return[ticker]=pd.Series((self.data[ticker][-1]-self.data[ticker][0])/self.data[ticker][0])
        annualized_return_raw=total_return.apply(lambda num: (((1 + num)**(1/year))-1))
        annualized_return_melt=annualized_return_raw.melt()["value"]
        annualized_return_melt.index=self.tickers
        return annualized_return_melt
    
    def std_df(self):
        std_df=self.data.pct_change().std()*np.sqrt(250)
        return vol_df
    
    def std_df_neg(self):
        df_clean_returns=self.data.pct_change()
        vol_df_neg=df_clean_returns[df_clean_returns < 0].std()*np.sqrt(250)     
        return vol_df_neg
    
#annualized_return_melt["value"]
#annualized_return=annualized_return(self.data,self.tickers,9)
#annualized_return.set_index(self.tickers,inplace=True)
#print(annualized_return)
returns=Returns(df_clean,tickers_clean)
annualized_returns=returns.annualized_return(9)
std_df=returns.std_df()
std_df_neg=returns.std_df_neg()

MMM     0.147203
ABT     0.137910
ABMD    0.353804
ACN     0.164488
ATVI    0.196466
          ...   
XLNX    0.188542
YUM     0.184078
ZBRA    0.254681
ZBH     0.164398
ZION    0.220402
Length: 452, dtype: float64
MMM     0.185263
ABT     0.184818
ABMD    0.459786
ACN     0.215679
ATVI    0.293198
          ...   
XLNX    0.264983
YUM     0.228975
ZBRA    0.326808
ZBH     0.222412
ZION    0.320216
Length: 452, dtype: float64


In [486]:
class RAR: 
    def __init__(self,rfr,annualized_return):
        self.rfr=rfr
        self.annualized_return=annualized_return

    def risk_ratio(self,std_df):
        sharpe_ratio=(self.annualized_return-self.rfr)/std_df
        return sharpe_ratio.sort_values()
    
    def risk_ratio_weights(self,std_df):
        sharpe_ratio=(self.annualized_return-self.rfr)/std_df
        sharpe_ratio_pos=sharpe_ratio[sharpe_ratio>0]
        sharpe_ratio_weights=sharpe_ratio_pos/sharpe_ratio_pos.sum()
        return sharpe_ratio_weights.sort_values()
    
    
rfr=0.01
RAR=RAR(rfr,annualized_returns)
sharpe_ratio=RAR.risk_ratio(std_df)
sortino_ratio=RAR.risk_ratio(std_df_neg)
print(sharpe_ratio)
print(sortino_ratio)
sharpe_ratio_weights=RAR.risk_ratio_weights(std_df)
sortino_ratio_weights=RAR.risk_ratio_weights(std_df_neg)
print(sharpe_ratio_weights)
print(sortino_ratio_weights)
#RAR.sharpe_ratio_weights()
#sharp_weights=sharp_ratio(0.01,annualized_return,vol_df)
#print(sharp_weights)

APA    -0.404462
DVN    -0.336875
FCX    -0.256887
ARNC   -0.226400
GE     -0.209121
          ...   
HD      1.194032
UNH     1.211593
EXR     1.233856
TDG     1.281542
NI      1.311717
Length: 452, dtype: float64
APA    -0.600727
DVN    -0.494947
FCX    -0.370553
ARNC   -0.309196
SLB    -0.294652
          ...   
HD      1.727471
MKTX    1.733992
EXR     1.774186
UNH     1.777863
NI      1.811911
Length: 452, dtype: float64
GS      0.000033
XRAY    0.000047
IBM     0.000084
TPR     0.000127
WU      0.000130
          ...   
HD      0.005108
UNH     0.005184
EXR     0.005279
TDG     0.005483
NI      0.005612
Length: 427, dtype: float64
GS      0.000032
XRAY    0.000042
IBM     0.000078
WU      0.000108
TPR     0.000115
          ...   
HD      0.005289
MKTX    0.005309
EXR     0.005432
UNH     0.005443
NI      0.005548
Length: 427, dtype: float64
